# MapReduce (I)

In [87]:
!wget https://aitor-medrano.github.io/iabd/spark/resources/el_quijote.txt
!mv ./el_quijote.txt ./pr0401
!hdfs dfs -mkdir /palabras
!hdfs dfs -put pr0401/el_quijote.txt /palabras
!hdfs dfs -ls /palabras

--2025-11-27 09:44:36--  https://aitor-medrano.github.io/iabd/spark/resources/el_quijote.txt
Resolving aitor-medrano.github.io (aitor-medrano.github.io)... 185.199.111.153, 185.199.108.153, 185.199.109.153, ...
Connecting to aitor-medrano.github.io (aitor-medrano.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1060259 (1.0M) [text/plain]
Saving to: ‘el_quijote.txt’

el_quijote.txt      100%[===================>]   1.01M  --.-KB/s    in 0.1s    

2025-11-27 09:44:37 (10.3 MB/s) - ‘el_quijote.txt’ saved [1060259/1060259]

mkdir: `/palabras': File exists
put: `/palabras/el_quijote.txt': File exists
Found 1 items
-rw-r--r--   3 root supergroup    1060259 2025-11-27 08:50 /palabras/el_quijote.txt


## Ejercicio 1

In [88]:
%%writefile pr0401/ejercicio1/mapper.py
#!/usr/bin/env python3
import sys
chars_to_remove = "\"!·$%&/()=?¿^*¨_:;,.-´`+¡'ºª\|@#~€¬{[]}\¸~ ̣·•• ´´“1234567890\t\n\r\a\b\f\r\t\v\\\n\r–‘‘«»"
for line in sys.stdin:
    words = line.split()
    for word in words:
        token = "".join(filter(lambda c: c not in chars_to_remove, word.lower()))
        if token:
            print(f"{token}\t1")

Overwriting pr0401/ejercicio1/mapper.py


In [89]:
%%writefile pr0401/ejercicio1/reducer.py
#!/usr/bin/env python3
import sys

cur_word = None
count = 0
for line in sys.stdin:
    word, _ = line.split("\t")
    if cur_word == word:
        count += 1
    else:
        if cur_word:
            print(f"{cur_word},{count}")
        cur_word = word
        count = 1
if cur_word:
    print(f"{cur_word},{count}")


Overwriting pr0401/ejercicio1/reducer.py


In [90]:
!cat pr0401/el_quijote.txt | python3 pr0401/ejercicio1/mapper.py | sort | python3 pr0401/ejercicio1/reducer.py

a,4821
aa,1
abad,1
abadejo,2
abades,1
abadesa,1
abajarse,2
abajen,1
abajo,21
abajó,1
abalanza,1
abalánzase,1
abandonarme,1
abatanar,1
abecé,3
abejas,1
abencerraje,1
abierta,4
abiertas,1
abierto,7
abiertos,5
abindarráez,3
abismo,3
ablandaba,1
ablande,1
ablandó,1
abollada,1
abollado,1
abominábamos,1
aborrascadas,1
aborrece,3
aborrecida,1
aborrecido,3
aborrecimiento,2
aborreció,1
aborrecí,1
aborrezco,3
abra,2
abracé,1
abran,3
abrasa,1
abrasada,1
abrasados,2
abrasar,1
abrasó,2
abrazada,5
abrazado,4
abrazamiento,1
abrazando,2
abrazar,9
abrazara,1
abrazará,1
abrazarle,1
abrazaron,1
abrazarse,1
abrazase,1
abrazándole,4
abrazándonos,1
abrazándose,1
abrazáronse,2
abrazo,1
abrazos,1
abrazó,7
abrazóla,1
abrazóle,1
abre,2
abreviar,2
abrevió,1
abrid,1
abriendo,9
abriera,2
abriese,1
abriéle,1
abriéndola,1
abriéndole,1
abriéndolos,1
abrigo,1
abrila,1
abrió,13
abrióle,1
abrióse,1
abrir,6
abrirá,1
abrirían,1
abrirla,1
abrirle,1
abrirse,2
abrí,1
abrían,2
abrumado,1
absoluta

In [91]:
!hdfs dfs -rm /temperaturas/salida_pr0401_ej1/*
!hdfs dfs -rmdir /temperaturas/salida_pr0401_ej1
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file pr0401/ejercicio1/mapper.py \
-mapper pr0401/ejercicio1/mapper.py \
-file pr0401/ejercicio1/reducer.py \
-reducer pr0401/ejercicio1/reducer.py \
-input /palabras/el_quijote.txt \
-output /temperaturas/salida_pr0401_ej1
!hdfs dfs -cat /temperaturas/salida_pr0401_ej1/part-00000

Deleted /temperaturas/salida_pr0401_ej1/_SUCCESS
Deleted /temperaturas/salida_pr0401_ej1/part-00000
2025-11-27 09:44:48,627 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [pr0401/ejercicio1/mapper.py, pr0401/ejercicio1/reducer.py, /tmp/hadoop-unjar1908913794543316685/] [] /tmp/streamjob2266377034541048964.jar tmpDir=null
2025-11-27 09:44:49,482 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.19.0.6:8032
2025-11-27 09:44:49,609 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.19.0.6:8032
2025-11-27 09:44:49,892 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1764233199108_0007
2025-11-27 09:44:50,396 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-27 09:44:50,553 INFO mapreduce.JobSubmitter: number of splits:2
2025-11-27 09:44:50,760 INFO map

## Ejercicio 2

In [92]:
%%writefile pr0401/ejercicio2/mapper.py
#!/usr/bin/env python3
import sys
chars_to_remove = "\"!·$%&/()=?¿^*¨_:;,.-´`+¡'ºª\|@#~€¬{[]}\¸~ ̣·•• ´´“1234567890\t\n\r\a\b\f\r\t\v\\\n\r–‘‘«»"
with open("palabras.txt") as file:
    content = file.read().split(",")
for line in sys.stdin:
    words = line.split()
    for word in words:
        token = "".join(filter(lambda c: c not in chars_to_remove, word.lower()))
        if token in content:
            continue
        if token:
            print(f"{token}\t1")

Overwriting pr0401/ejercicio2/mapper.py


In [93]:
!hdfs dfs -rm /temperaturas/salida_pr0401_ej2/*
!hdfs dfs -rmdir /temperaturas/salida_pr0401_ej2
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file pr0401/ejercicio2/palabras.txt \
-file pr0401/ejercicio2/mapper.py \
-mapper pr0401/ejercicio2/mapper.py \
-file pr0401/ejercicio1/reducer.py \
-reducer pr0401/ejercicio1/reducer.py \
-input /palabras/el_quijote.txt \
-output /temperaturas/salida_pr0401_ej2
!hdfs dfs -cat /temperaturas/salida_pr0401_ej2/part-00000

Deleted /temperaturas/salida_pr0401_ej2/_SUCCESS
Deleted /temperaturas/salida_pr0401_ej2/part-00000
2025-11-27 09:45:16,554 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [pr0401/ejercicio2/palabras.txt, pr0401/ejercicio2/mapper.py, pr0401/ejercicio1/reducer.py, /tmp/hadoop-unjar2813315308230209637/] [] /tmp/streamjob2921740743873532828.jar tmpDir=null
2025-11-27 09:45:17,865 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.19.0.6:8032
2025-11-27 09:45:18,150 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.19.0.6:8032
2025-11-27 09:45:18,561 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1764233199108_0008
2025-11-27 09:45:19,268 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-27 09:45:19,440 INFO mapreduce.JobSubmitter: number of splits:2


## Ejercicio 3

In [111]:
%%writefile pr0401/ejercicio3/mapper.py
#!/usr/bin/env python3

import sys
for line in sys.stdin:
    word, count = line.split(",")
    print(f"{1/int(count)},{word}")


Overwriting pr0401/ejercicio3/mapper.py


In [114]:
%%writefile pr0401/ejercicio3/reducer.py
#!/usr/bin/env python3

import sys
for line in sys.stdin:
    count, word = line.split(",")
    print(f"{word},{int(1/float(count))}")



Overwriting pr0401/ejercicio3/reducer.py


In [115]:
!hdfs dfs -rm /temperaturas/salida_pr0401_ej3/*
!hdfs dfs -rmdir /temperaturas/salida_pr0401_ej3
!hdfs dfs -rm /temperaturas/salida_pr0401_ej3.1/*
!hdfs dfs -rmdir /temperaturas/salida_pr0401_ej3.1
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file pr0401/ejercicio2/palabras.txt \
-file pr0401/ejercicio2/mapper.py \
-mapper pr0401/ejercicio2/mapper.py \
-file pr0401/ejercicio1/reducer.py \
-reducer pr0401/ejercicio1/reducer.py \
-input /palabras/el_quijote.txt \
-output /temperaturas/salida_pr0401_ej3
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file pr0401/ejercicio3/mapper.py \
-mapper pr0401/ejercicio3/mapper.py \
-file pr0401/ejercicio3/reducer.py \
-reducer pr0401/ejercicio3/reducer.py \
-input /temperaturas/salida_pr0401_ej3/part-00000 \
-output /temperaturas/salida_pr0401_ej3.1


Deleted /temperaturas/salida_pr0401_ej3/_SUCCESS
Deleted /temperaturas/salida_pr0401_ej3/part-00000
Deleted /temperaturas/salida_pr0401_ej3.1/_SUCCESS
Deleted /temperaturas/salida_pr0401_ej3.1/part-00000
^C
2025-11-27 10:00:56,500 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [pr0401/ejercicio2/palabras.txt, pr0401/ejercicio2/mapper.py, pr0401/ejercicio1/reducer.py, /tmp/hadoop-unjar431006359811026991/] [] /tmp/streamjob1267148173692491806.jar tmpDir=null
2025-11-27 10:00:56,927 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.19.0.6:8032
2025-11-27 10:00:57,032 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.19.0.6:8032
2025-11-27 10:00:57,160 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1764233199108_0018
2025-11-27 10:00:57,432 INFO mapred.FileInputFormat:

In [116]:
!hdfs dfs -cat /temperaturas/salida_pr0401_ej3.1/part-00000

don	
,1073	
más	
,969	
quijote	
,845	
él	
,749	
sancho	
,691	
había	
,672	
así	
,575	
respondió	
,448	
señor	
,419	
merced	
,400	
qué	
,380	
caballero	
,352	
cosa	
,281	
allí	
,266	
mal	
,253	
cura	
,245	
señora	
,241	
aquí	
,225	
tenía	
,219	
mas	
,213	
está	
,202	
dios	
,201	
vida	
,200	
padre	
,196	
sé	
,185	
mundo	
,170	
puesto	
,168	
amo	
,162	
digo	
,159	
caballeros	
,155	
cómo	
,153	
camila	
,150	
mí	
,150	
tierra	
,150	
casa	
,148	
mano	
,148	
aun	
,147	
habían	
,142	
lotario	
,142	
algún	
,140	
barbero	
,138	
anselmo	
,137	
cielo	
,137	
visto	
,135	
amigo	
,133	
tú	
,132	
decía	
,131	
dorotea	
,131	
ojos	
,128	
punto	
,127	
día	
,126	
también	
,126	
sólo	
,125	
quién	
,124	
podía	
,122	
panza	
,122	
camino	
,120	
después	
,120	
rocinante	
,120	
hombre	
,119	
fernando	
,118	
razones	
,116	
días	
,115	
sí	
,115	
gusto	
,114	
fuese	
,112	
voluntad	
,112	
será	
,111	
venta	
,111	
libros	
,110	
manos	
,110	
quiero	
,110	
vio	
,110	
noche	
,107	
escudero	